In [1]:
import datetime
import glob
import os

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy.stats as stats
import tensorflow as tf
from tqdm import tqdm

import utils

In [19]:
chour_dict = {'morning':'0400',
              'afternoon':'1200',
             }

ampm = 'morning'

width = 8

critical_hour = chour_dict[ampm]

df = pd.read_csv(f'merged_df_{ampm}.csv',parse_dates=['time'])
time = df['time']
df = df.drop(['time','tmpf','tmpf_pred'],axis=1)

In [20]:
(train_df,val_df,test_df), num_features = utils.train_test_val_split(df)

In [21]:
day_list = []
for group in df.groupby(time.dt.date):
    if ((group[1]['lt_32']>1) & (group[1]['p01i']>0)).any():
        day_list.append(group[1])

In [22]:
len(day_list)

16

In [23]:
lstm_no = tf.keras.models.load_model(f'no_weather_{ampm}')
lstm_wx = tf.keras.models.load_model(f'weather_{ampm}')

In [24]:
mn = pd.read_csv('mean.csv')
mn = mn.set_index('Unnamed: 0').T
std = pd.read_csv('std.csv')
std = std.set_index('Unnamed: 0').T

In [103]:
precip_thresh = 0.125
precip_thresh = precip_thresh**(1/3)
precip_thresh

0.4641588833612779

In [114]:
tem = val_df[(val_df['p01i']>precip_thresh)]

In [115]:
time_test = time[tem.index]

In [116]:
forecast_starts = pd.date_range(f'2015-01-01 {critical_hour}',f'2019-12-31 {critical_hour}',freq='24h')
# valid_inds = time_test[time_test.isin(forecast_starts)].index
valid_inds = time_test[(time_test.dt.hour<10) & (time_test.dt.hour>4)].index

In [117]:
def no_norm(value,name,ampm):
    mn = pd.read_csv(f'mean_{ampm}.csv',index_col=0)
    mn = mn.T
    std = pd.read_csv(f'std_{ampm}.csv',index_col=0)
    std = std.T
    return (value*std)[name].values[0]

In [118]:
rmse = np.ones(len(valid_inds))
val = np.ones((len(valid_inds),8))*np.nan
pred = np.ones((len(valid_inds),8))*np.nan


for ind_val,ind in enumerate(valid_inds):
        pred_i = lstm_wx.predict(df[ind:ind+width].values[np.newaxis,])
        pred_i = pred_i[0,:,1]
        val_i = df[ind+width:ind+2*width]['speed']
        if len(val_i) > 7:
            rmse[ind_val] = np.sqrt(((val_i.values-pred_i.T)**2).mean())

print(no_norm(rmse.mean(),'speed',mn,std))

2.0861040033589773


In [119]:
df_no = df.drop(['lt_32','p01i','p01i_pred'],axis=1)

In [120]:
def no_norm(value,name,mn,std):
    return (value*std)[name].values[0]

In [121]:
rmse = np.ones(len(valid_inds))
val = np.ones((len(valid_inds),8))*np.nan
pred = np.ones((len(valid_inds),8))*np.nan

for ind_val,ind in enumerate(valid_inds):
        pred_i = lstm_no.predict(df_no[ind:ind+width].values[np.newaxis,])
        pred_i = pred_i[0,:,1]
        val_i = df_no[ind+width:ind+2*width]['speed']
        if len(val_i) > 7:
            rmse[ind_val] = np.sqrt(((val_i.values-pred_i.T)**2).mean())
    
print(no_norm(rmse.mean(),'speed',mn,std))

2.402131362355975
